In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
from matplotlib import cm

In [128]:
all_files  = glob.glob(r'/share/gpu0/jjwhit/kappa_cosmos_simulations/cropped_dataset/kappa_train/*.npy')

In [18]:
len(all_files)


8500

In [138]:
#GIF of  simulations
images = []
frames = [np.load(file) for file in all_files[:100]]
# Normalize and scale the float64 values to uint8 in the range [0, 255]
scaled_frames = [(frame - np.min(frame)) / (np.max(frame) - np.min(frame)) * 255 for frame in frames]
uint8_frames = [np.uint8(frame) for frame in scaled_frames]
coloured_frames  = [np.uint8(cm.magma(frame)*255) for frame in uint8_frames]

#imgs = np.stack(uint8_frames)
imgs = np.stack(coloured_frames)

# Save the GIF with the desired parameters
Image.fromarray((imgs[0])).save("/share/gpu0/jjwhit/plots/simulations.gif", save_all=True, append_images=[Image.fromarray(img) for img in imgs[1:]], duration=250, loop=0)


In [3]:
#GIF of cosmos posterior samples
cosmos_samps = np.load('/home/jjwhit/rcGAN/mass_map_utils/cosmos/cosmos_samps_1k.npy', allow_pickle=True)


In [6]:
cosmos_samps

In [ ]:
images = []
frames = [cosmos_samps[:, :, i] for i in range(cosmos_samps.shape[2])]
scaled_frames = [(frame - np.min(frame)) / (np.max(frame) - np.min(frame)) * 255 for frame in frames]
uint8_frames = [np.uint8(frame) for frame in scaled_frames]
coloured_frames  = [np.uint8(cm.magma(frame)*255) for frame in uint8_frames]

#imgs = np.stack(uint8_frames)
imgs = np.stack(coloured_frames)
Image.fromarray((imgs[0])).save("/share/gpu0/jjwhit/plots/cosmos_samples.gif", save_all=True, append_images=[Image.fromarray(img) for img in imgs[1:]], duration=100, loop=0)